# 📊 Análise Exploratória de Dados - SPTrans Pipeline

**Autor:** Rafael - SPTrans Data Team  
**Data:** Janeiro 2025  
**Objetivo:** Explorar e entender os dados da API SPTrans e GTFS

---

## Sumário
1. [Setup e Imports](#1-setup)
2. [Análise da API de Posições](#2-api-positions)
3. [Análise dos Dados GTFS](#3-gtfs-data)
4. [Qualidade dos Dados](#4-data-quality)
5. [Visualizações](#5-visualizations)
6. [Insights e Conclusões](#6-insights)

## 1. Setup e Imports {#1-setup}

In [ ]:
# Imports básicos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurar visualizações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Configurar pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Imports concluídos")

In [ ]:
# Imports do projeto
import sys
sys.path.append('..')

from src.common.config import Config
from src.ingestion.sptrans_api_client import SPTransClient
from src.common.validators import CoordinateValidator, DataQualityValidator

config = Config()
print("✅ Módulos do projeto importados")

## 2. Análise da API de Posições {#2-api-positions}

In [ ]:
# Conectar à API
client = SPTransClient(token=config.SPTRANS_API_TOKEN)

# Obter posições atuais
print("Obtendo posições dos veículos...")
positions_data = client.get_all_positions()

print(f"✅ Dados obtidos: {len(positions_data.get('l', []))} linhas")

In [ ]:
# Processar dados em DataFrame
vehicles_list = []

for line in positions_data.get('l', []):
    route_code = line.get('c')
    route_name = line.get('lt0', '')
    
    for vehicle in line.get('vs', []):
        vehicles_list.append({
            'vehicle_id': vehicle.get('p'),
            'route_code': route_code,
            'route_name': route_name,
            'latitude': vehicle.get('py'),
            'longitude': vehicle.get('px'),
            'has_accessibility': vehicle.get('a', False),
            'timestamp': vehicle.get('ta')
        })

df_positions = pd.DataFrame(vehicles_list)

print(f"✅ DataFrame criado: {len(df_positions):,} veículos")
df_positions.head()

In [ ]:
# Estatísticas básicas
print("📊 ESTATÍSTICAS BÁSICAS")
print("="*60)
print(f"Total de veículos: {len(df_positions):,}")
print(f"Total de rotas: {df_positions['route_code'].nunique():,}")
print(f"Veículos com acessibilidade: {df_positions['has_accessibility'].sum():,} ({df_positions['has_accessibility'].mean()*100:.1f}%)")
print(f"\nDados por rota:")
print(df_positions.groupby('route_code').size().sort_values(ascending=False).head(10))

## 3. Análise dos Dados GTFS {#3-gtfs-data}

In [ ]:
# Carregar dados GTFS
print("Carregando dados GTFS...")

gtfs_path = '../data/gtfs/'

# Routes
df_routes = pd.read_csv(f'{gtfs_path}routes.txt')
print(f"✅ Routes: {len(df_routes):,} linhas")

# Stops
df_stops = pd.read_csv(f'{gtfs_path}stops.txt')
print(f"✅ Stops: {len(df_stops):,} paradas")

# Trips
df_trips = pd.read_csv(f'{gtfs_path}trips.txt')
print(f"✅ Trips: {len(df_trips):,} viagens")

In [ ]:
# Análise de routes
print("📊 ANÁLISE DE ROTAS (GTFS)")
print("="*60)
print(f"Total de rotas: {len(df_routes):,}")
print(f"\nTipos de rota:")
print(df_routes['route_type'].value_counts())
print(f"\nTop 10 rotas por nome:")
df_routes.head(10)[['route_short_name', 'route_long_name']]

In [ ]:
# Análise de stops (paradas)
print("📊 ANÁLISE DE PARADAS (GTFS)")
print("="*60)
print(f"Total de paradas: {len(df_stops):,}")

# Distribuição geográfica
print(f"\nDistribuição Geográfica:")
print(f"Latitude: {df_stops['stop_lat'].min():.4f} a {df_stops['stop_lat'].max():.4f}")
print(f"Longitude: {df_stops['stop_lon'].min():.4f} a {df_stops['stop_lon'].max():.4f}")

# Amostra
df_stops.head()

## 4. Qualidade dos Dados {#4-data-quality}

In [ ]:
# Validar coordenadas das posições
print("🔍 VALIDAÇÃO DE QUALIDADE - POSIÇÕES")
print("="*60)

# Verificar coordenadas válidas
valid_coords = df_positions.apply(
    lambda row: CoordinateValidator.is_valid_coordinate(row['latitude'], row['longitude']),
    axis=1
)

print(f"Coordenadas válidas: {valid_coords.sum():,} ({valid_coords.mean()*100:.2f}%)")
print(f"Coordenadas inválidas: {(~valid_coords).sum():,}")

# Valores nulos
print(f"\nValores nulos:")
print(df_positions.isnull().sum())

# Duplicatas
duplicates = df_positions.duplicated(subset=['vehicle_id']).sum()
print(f"\nDuplicatas (vehicle_id): {duplicates:,}")

In [ ]:
# Validar paradas GTFS
print("🔍 VALIDAÇÃO DE QUALIDADE - PARADAS (GTFS)")
print("="*60)

# Verificar coordenadas válidas
valid_stops = df_stops.apply(
    lambda row: CoordinateValidator.is_valid_coordinate(row['stop_lat'], row['stop_lon'], strict=False),
    axis=1
)

print(f"Paradas com coordenadas válidas: {valid_stops.sum():,} ({valid_stops.mean()*100:.2f}%)")
print(f"Paradas com coordenadas inválidas: {(~valid_stops).sum():,}")

# Valores nulos
print(f"\nValores nulos nas paradas:")
print(df_stops.isnull().sum())

## 5. Visualizações {#5-visualizations}

In [ ]:
# Distribuição de veículos por rota (top 20)
fig, ax = plt.subplots(figsize=(14, 6))

top_routes = df_positions['route_code'].value_counts().head(20)
top_routes.plot(kind='bar', ax=ax, color='steelblue')

ax.set_title('Top 20 Rotas com Mais Veículos', fontsize=16, fontweight='bold')
ax.set_xlabel('Código da Rota', fontsize=12)
ax.set_ylabel('Número de Veículos', fontsize=12)
ax.grid(axis='y', alpha=0.3)

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print(f"✅ Visualização gerada: Top 20 rotas")

In [ ]:
# Mapa de calor - Distribuição geográfica de veículos
fig, ax = plt.subplots(figsize=(12, 10))

# Plotar posições dos veículos
scatter = ax.scatter(
    df_positions['longitude'],
    df_positions['latitude'],
    c='red',
    alpha=0.3,
    s=10,
    label='Veículos'
)

# Plotar paradas
ax.scatter(
    df_stops['stop_lon'],
    df_stops['stop_lat'],
    c='blue',
    alpha=0.1,
    s=2,
    label='Paradas'
)

ax.set_title('Distribuição Geográfica - Veículos e Paradas (São Paulo)', fontsize=16, fontweight='bold')
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✅ Mapa de distribuição gerado")

In [ ]:
# Acessibilidade dos veículos
fig, ax = plt.subplots(figsize=(8, 6))

accessibility_counts = df_positions['has_accessibility'].value_counts()
colors = ['#ff6b6b', '#4ecdc4']
labels = ['Sem Acessibilidade', 'Com Acessibilidade']

ax.pie(
    accessibility_counts,
    labels=labels,
    autopct='%1.1f%%',
    colors=colors,
    startangle=90,
    textprops={'fontsize': 12}
)

ax.set_title('Distribuição de Acessibilidade nos Veículos', fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"✅ Gráfico de acessibilidade gerado")

## 6. Insights e Conclusões {#6-insights}

In [ ]:
print("📝 INSIGHTS E CONCLUSÕES")
print("="*80)

print(f"\n1️⃣ VOLUME DE DADOS")
print(f"   • {len(df_positions):,} veículos ativos no momento da coleta")
print(f"   • {df_positions['route_code'].nunique():,} rotas diferentes em operação")
print(f"   • {len(df_stops):,} paradas cadastradas no sistema GTFS")

print(f"\n2️⃣ QUALIDADE DOS DADOS")
valid_pct = valid_coords.mean() * 100
print(f"   • {valid_pct:.2f}% das coordenadas são válidas (dentro de São Paulo)")
print(f"   • Taxa de nulos: {df_positions.isnull().sum().sum() / df_positions.size * 100:.2f}%")

print(f"\n3️⃣ ACESSIBILIDADE")
accessibility_pct = df_positions['has_accessibility'].mean() * 100
print(f"   • {accessibility_pct:.1f}% dos veículos possuem acessibilidade")

print(f"\n4️⃣ DISTRIBUIÇÃO GEOGRÁFICA")
print(f"   • Veículos distribuídos por toda área metropolitana")
print(f"   • Maior concentração no centro expandido")

print(f"\n5️⃣ RECOMENDAÇÕES")
print(f"   ✅ Dados têm qualidade suficiente para análises")
print(f"   ✅ API fornece dados em tempo real consistentes")
print(f"   ⚠️  Validação de coordenadas é essencial")
print(f"   ⚠️  Tratamento de valores nulos necessário")

print("\n" + "="*80)

---

## 📌 Próximos Passos

1. **Análise temporal**: Coletar dados ao longo do tempo para identificar padrões
2. **Performance**: Análise de velocidades e tempos de viagem
3. **Headway**: Calcular intervalos entre ônibus
4. **Integração**: Cruzar dados de API com GTFS para enriquecimento
5. **ML**: Modelos preditivos de atrasos e demanda

---

**Notebook criado por:** Rafael - SPTrans Data Team  
**Data:** Janeiro 2025  
**Versão:** 1.0